In [ ]:
# REST
from coinbase.rest import RESTClient
import pandas as pd
import time

client = RESTClient(api_key=api_key, api_secret=api_secret)

# Function to fetch order book data using get_product_book with specified limit
def get_order_book(product_id, limit):
    response = client.get_product_book(product_id=product_id, limit=limit)
    
    # Extracting bids and asks from the 'pricebook' part of the response
    pricebook = response.get('pricebook', {})
    
    bids = pd.DataFrame(pricebook.get('bids', []))
    asks = pd.DataFrame(pricebook.get('asks', []))
    
    # Adding time column to each dataframe
    time = pricebook.get('time', 'N/A')
    bids['time'] = time
    asks['time'] = time

    # Renaming columns to differentiate bids and asks
    bids.columns = ['bid_price', 'bid_size', 'time']
    asks.columns = ['ask_price', 'ask_size', 'time']

    # Merging bids and asks into a single dataframe-like structure
    order_book_df = pd.concat([bids, asks], axis=1)
    
    return order_book_df

# Product ID and limit for order book levels
product_id = "BTC-USD"
limit = 10  # Fetch 10 levels of the order book

# Initialize an empty list to store all the dataframes
all_dfs = []

# Run the loop 1800 times (30 minutes with a 1-second interval)
for i in range(1800):
    df = get_order_book(product_id, limit)
    all_dfs.append(df)
    
    # Print individual dataframe for each iteration
    print(f"\nIteration {i+1} - Order Book Data at time: {df['time'].iloc[0]}")
    print(df.to_string(index=False))
    
    # Sleep for 1 second before the next API call
    time.sleep(1)

# Concatenate all the dataframes into one
final_df = pd.concat(all_dfs, ignore_index=True)

# Display final dataframe (optional, as it can be large)
print("\nFinal DataFrame after 1800 iterations:")
print(final_df)

# Save the final dataframe to a CSV file for future analysis
final_df.to_csv("order_book_data.csv", index=False)


In [ ]:
import pandas as pd

# Load the CSV file
file_path = 'order_book_data.csv'
lob_data = pd.read_csv(file_path)

# Add 'bid_level' and 'ask_level' columns based on the timestamp grouping
lob_data['bid_level'] = lob_data.groupby('time').cumcount(ascending=True) + 1
lob_data['ask_level'] = lob_data['bid_level']

lob_data.to_csv('LOB_modified.csv')


# Display the first few rows to verify the changes
print(lob_data.head(10))


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

file_path = 'LOB_modified.csv'
data = pd.read_csv(file_path)

# Define the number of timestamps you want to plot
num_timestamps = 50  # You can change this to any number to scale the plot

timestamps = data['time'].unique()[:num_timestamps]

# Extract the 10 levels for each of the timestamps
timestamp_data_list = [data[data['time'] == ts].head(10) for ts in timestamps]

# Create a list to store the level 10 bid and ask prices for each timestamp
level_10_prices = []

# Find the level 10 bid and ask prices for each timestamp
for ts_data in timestamp_data_list:
    level_10_bid_price = ts_data[ts_data['bid_level'] == 10]['bid_price'].iloc[0]
    level_10_ask_price = ts_data[ts_data['ask_level'] == 10]['ask_price'].iloc[0]
    level_10_prices.append(level_10_bid_price)
    level_10_prices.append(level_10_ask_price)

# Update the y-axis range based on the level 10 prices
min_price_level_10_corrected = min(level_10_prices)
max_price_level_10_corrected = max(level_10_prices)
y_min_final_corrected = min_price_level_10_corrected - (max_price_level_10_corrected - min_price_level_10_corrected) * 0.05
y_max_final_corrected = max_price_level_10_corrected + (max_price_level_10_corrected - min_price_level_10_corrected) * 0.05

sns.set(style="whitegrid")

plt.figure(figsize=(24, 14), dpi=300)  # Increase figure size and resolution

# Create the plot
fig, ax = plt.subplots(figsize=(24, 14))  # Increased size of the plot

# Store the mid prices for each timestamp to plot them as dots
mid_prices = []

# Iterate over each timestamp to create the time series plot
for i, ts_data in enumerate(timestamp_data_list):
    # Sort data by price for better visualization
    ts_data_sorted = ts_data.sort_values(by=['bid_price', 'ask_price'])

    # Correct mid price using bid and ask prices at level 1 for each timestamp
    level_1_bid_price = ts_data[ts_data['bid_level'] == 1]['bid_price'].iloc[0]
    level_1_ask_price = ts_data[ts_data['ask_level'] == 1]['ask_price'].iloc[0]
    mid_price = (level_1_bid_price + level_1_ask_price) / 2
    mid_prices.append(mid_price)

    # Prepare bid and ask data for heatmap
    bid_prices = ts_data_sorted['bid_price'].values
    ask_prices = ts_data_sorted['ask_price'].values
    bid_sizes = ts_data_sorted['bid_size'].values
    ask_sizes = ts_data_sorted['ask_size'].values

    # Concatenate bids and asks for plotting
    all_prices = np.concatenate((bid_prices, ask_prices))
    all_sizes = np.concatenate((bid_sizes, ask_sizes))

    # Create scatter plot for each timestamp
    x_values = np.full(len(all_prices), i + 1)  # x-values represent each timestamp (1, 2, ..., num_timestamps)
    ax.scatter(x_values, all_prices, c=all_sizes, cmap='viridis', s=200, edgecolor='black', linewidth=1.5)

# Plot mid prices as dots and connect them with a dotted line
ax.plot(range(1, num_timestamps + 1), mid_prices, marker='o', color='red', linestyle='--', markersize=10, linewidth=2.5, label='Mid Prices')

# Set axis labels and title with better fonts and spacing
ax.set_ylabel('Prices', fontsize=18)
ax.set_xlabel('Time', fontsize=18)
ax.set_title(f'LOB States', fontsize=22, pad=20)

# Set x-axis labels to the actual timestamps
ax.set_xticks(range(1, num_timestamps + 1))
ax.set_xticklabels(timestamps, rotation=45, ha='right', fontsize=12)

# Set the corrected y-axis range based on level 10 prices
ax.set_ylim([y_min_final_corrected, y_max_final_corrected])

# Add gridlines for better readability
ax.grid(True, linestyle='--', alpha=0.7)

# Add colorbar to represent volume (sizes)
cbar = plt.colorbar(ax.collections[0])
cbar.set_label('Volume', fontsize=16)

# Customize the legend
ax.legend(fontsize=14, loc='upper center')

plt.show()
